# Refugees in the Age of Gloabl Warming
### Data Engineering Capstone Project

#### Project Summary
This project focuses on monitoring refugee and population information around the world based on temperature changes over time.

The project follows the following steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [2]:
# Do all imports and installs here
import pandas as pd
import os
from datetime import datetime
import psycopg2
import configparser
config = configparser.ConfigParser()
from sqlalchemy.engine import create_engine

In [3]:
# Read in parameters needed for Redshift cluster
# config = configparser.ConfigParser()
# config.read('dwh.cfg')

# # Connect to Redshift cluster and gets cursor to it
# conn = psycopg2.connect("""host={} 
#                            dbname={} 
#                            user={} 
#                            password={} 
#                            port={}""".format(*config['CLUSTER'].values()))
# cur = conn.cursor()

### Step 1: Scope the Project and Gather Data

#### Scope 
What is your end solution look like? What tools did you use? etc>

The plan is to build a data warehouse for analytical processes, so analysts can design recurring and ad hoc reports over time using SQL. There is a strong emphasis in ensuring the warehouse is easy to interpret, performant, and quality assured.
 
#### Data Sources and Content

There are four source datasets:
 1. City_temperature.csv
     - Summary: average daily temperature for all major cities in the world from 1995 - 2020
     - Source: University of Dayton - separate txt files available for each city [here](https://academic.udayton.edu/kissock/http/Weather/default.htm). The data is available for research and non-commercial purposes only. Refer to [this page](https://academic.udayton.edu/kissock/http/Weather/default.htm) for license.
     - Secondary source: SRK via Kaggle - [link](https://www.kaggle.com/sudalairajkumar/daily-temperature-of-major-cities)
 2. Country_population_total_long.csv
     - Summary: annual population counts by country from 1960 - 2017
     - Source: The World Bank - [link](https://data.worldbank.org/indicator/SP.POP.TOTL)
     - Secondary source: Devakumar kp via Kaggle - [link](https://www.kaggle.com/imdevskp/world-population-19602018?select=population_total_long.csv)
 3. UNdata_City_Population_20210315.csv
     - Summary: annual population counts by city from 1970 - 2020 (contains gaps in 1970's)
     - Source: UN Data - [link](https://data.un.org/Data.aspx?d=POP&f=tableCode%3A240)
 4. UNdata_Refugees_20210217.csv
     - Summary: annual refugee counts from 1975 - 2016 by country of residence and country of origin
     - Source: UN Data - [link](http://data.un.org/Data.aspx?d=UNHCR&f=indID%3aType-Ref)

### Read in Each Dataset

#### Temperature Data

In [3]:
temp_df = pd.read_csv('Data/temperature_data/city_temperature.csv', engine = 'python')
temp_df.head()

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


#### Population Counts by Country and Year

In [4]:
country_pop_df = pd.read_csv('Data/country_population_data/country_population_total_long.csv', engine = 'python')
country_pop_df.head()

,Country Name,Year,Count
0,Aruba,1960,54211
1,Afghanistan,1960,8996973
2,Angola,1960,5454933
3,Albania,1960,1608800
4,Andorra,1960,13411


#### Population Counts by City and Year

In [5]:
city_pop_df = pd.read_csv('Data/city_population_data/UNdata_City_Population_20210315.csv', engine = 'python')
city_pop_df.head()

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Åland Islands,2019,Total,Both Sexes,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2020,11711.0,1
1,Åland Islands,2019,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2020,5606.0,1
2,Åland Islands,2019,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2020,6105.0,1
3,Åland Islands,2018,Total,Both Sexes,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2019,11709.0,1
4,Åland Islands,2018,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2019,5620.5,1


#### Refugee Counts by Year, Country of Residence, and Country of Origin

In [6]:
refugee_df = pd.read_csv('Data/refugee_data/UNdata_Refugees_20210317.csv', engine = 'python')
refugee_df.head()

,Country or territory of asylum or residence,Country or territory of origin,Year,Refugees,Refugees assisted by UNHCR,Total refugees and people in refugee-like situations,Total refugees and people in refugee-like situations assisted by UNHCR
0,Afghanistan,Iraq,2016,1.0,1.0,1.0,1.0
1,Afghanistan,Islamic Rep. of Iran,2016,33.0,33.0,33.0,33.0
2,Afghanistan,Pakistan,2016,59737.0,59737.0,59737.0,59737.0
3,Albania,China,2016,11.0,11.0,11.0,11.0
4,Albania,Dem. Rep. of the Congo,2016,3.0,3.0,3.0,3.0


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

#### Temperature Data

In [7]:
# Add a date field
temp_df['Date'] = temp_df['Month'].map(str) + '/' + \
                  temp_df['Day'].map(str) + '/' +  \
                  temp_df['Year'].map(str)

# Summary stats by region
temp_df[['Region', 'Date', 'AvgTemperature']] \
    .groupby(['Region']) \
    .agg(['min', 'max', 'nunique'])
# Note that there are seven regions

Date                   AvgTemperature  \
                                        min       max nunique            min   
Region                                                                         
Africa                             1/1/1995  9/9/2019    9297          -99.0   
Asia                               1/1/1995  9/9/2019    9265          -99.0   
Australia/South Pacific            1/1/1995  9/9/2019    9265          -99.0   
Europe                             1/1/1995  9/9/2019    9356          -99.0   
Middle East                        1/1/1995  9/9/2019    9265          -99.0   
North America                      1/1/1995  9/9/2019    9295          -99.0   
South/Central America & Carribean  1/1/1995  9/9/2019    9266          -99.0   

                                                  
                                     max nunique  
Region                                            
Africa                             102.8     654  
Asia                               103.7    1334  
Australia/South Pacific             96.8     600  
Europe                             102.5    1079  
Middle East                        110.0     996  
North America                      107.7    1474  
South/Central America & Carribean   97.4     609

In [8]:
# Check how consistent information is populated by city over time
temp_df.groupby(['Year'])[['City']].nunique()
# Very consistent going back to 1995

,City
Year,
200,2
201,7
1995,319
1996,319
1997,320
1998,321
1999,321
2000,321
2001,321


In [9]:
# Note: there are records with a day value of 0. Exclude.
temp_df = temp_df[temp_df.Day > 0]

# Note: there are records with a year value of 200 and 201. Exclude.
temp_df = temp_df[temp_df.Year >= 1995]

# Note: missing temps are represented as -99. Exclude.
temp_df = temp_df[temp_df.AvgTemperature != -99]

# Drop duplicates
temp_df = temp_df.drop_duplicates()

# Verify there are no duplicate records at the most granular level, which is date and city
# 94 duplicates remain. Need to research further.
duplicate = temp_df[temp_df.duplicated(['State', 'City', 'Date'], keep=False)] \
                        .sort_values(by = ['City', 'Date'])
duplicate.head(20)

,Region,Country,State,City,Month,Day,Year,AvgTemperature,Date
744576,Europe,Germany,NaN,Hamburg,1,25,2011,38.5,1/25/2011
744970,Europe,Germany,NaN,Hamburg,1,25,2011,38.6,1/25/2011
743935,Europe,Germany,NaN,Hamburg,5,25,2010,51.7,5/25/2010
744330,Europe,Germany,NaN,Hamburg,5,25,2010,52.0,5/25/2010
743962,Europe,Germany,NaN,Hamburg,6,21,2010,57.4,6/21/2010
744357,Europe,Germany,NaN,Hamburg,6,21,2010,58.6,6/21/2010
754882,Europe,Germany,NaN,Munich,1,15,2019,35.0,1/15/2019
755249,Europe,Germany,NaN,Munich,1,15,2019,35.1,1/15/2019
754151,Europe,Germany,NaN,Munich,1,16,2018,41.5,1/16/2018
754518,Europe,Germany,NaN,Munich,1,16,2018,40.9,1/16/2018


In [11]:
# Summary stats by city or country for a region of interest.
# Note: there are some time gaps in cities.
def summary_stats(GroupBy, Region):
    return temp_df[[GroupBy, 'Date', 'AvgTemperature']].where(temp_df.Region == Region) \
               .groupby([GroupBy]) \
               .agg(['min', 'max', 'nunique'])

# GroupBy input options: City, Country, State
# Region input options: Africa, Asia, Australia/South Pacific, Europe, Middle East, North America, South/Central America & Carribean
summary_stats(GroupBy = 'City', Region = 'South/Central America & Carribean')

Date                   AvgTemperature              
                     min       max nunique            min   max nunique
City                                                                   
Belize City     1/1/1996  9/9/2019    8867           64.6  92.9     233
Bogota          1/1/1995  9/9/2019    9196           46.7  66.7     154
Brasilia        1/1/1995  9/9/2019    9206           56.1  87.7     244
Bridgetown      1/1/1995  9/9/2016    8349           74.2  88.0     120
Buenos Aires    1/1/1995  9/9/2019    9231           35.3  90.9     507
Caracas         1/1/1995  9/9/2019    9121           71.5  89.9     162
Georgetown      1/1/1999  9/9/2006    2136           67.0  90.6     135
Guatemala City  1/1/1997  9/9/2019    8710           51.2  79.8     220
Guayaquil       1/1/1995  9/9/2019    9036           67.2  90.0     188
Hamilton        1/1/1995  9/9/2009    5558           51.1  85.4     307
Havana          1/1/1996  9/9/2019    9094           46.9  88.3     301
La Paz          1/1/1995  9/9/2019    9225           32.8  63.4     211
Lima            1/1/1995  9/9/2015    7606           57.5  81.8     235
Managua         1/1/1997  9/9/2019    8737           68.5  93.9     180
Montvideo       1/1/1995  9/9/2019    9229           35.7  87.4     449
Nassau          1/1/1995  9/9/2019    9161           58.7  91.8     293
Panama City     1/1/1995  9/9/2019    9192           73.4  90.6     144
Paramaribo      1/1/1995  9/9/2019    7599           71.6  90.5     151
Port au Prince  1/1/2002  9/9/2019    5816           71.4  97.4     200
Quito           1/1/1995  9/9/2012    6594           49.1  69.0     163
Rio de Janeiro  1/1/1995  9/9/2019    9116           60.0  93.4     282
San Jose        1/1/1995  9/9/2019    9130           63.1  85.6     160
Santo Domingo   1/1/1996  9/9/2019    8513           65.0  87.4     170
Sao Paulo       1/1/1995  9/9/2019    9174           44.8  89.2     347
Tegucigalpa     1/1/1995  9/9/2019    9095           56.1  88.0     270

In [12]:
# Verify location content naming conventions are consistent to avoid separating temperatures that belong together
def unique_values(Region):
    return temp_df[['Region', 'Country', 'State', 'City']].where(temp_df.Region == Region) \
               .drop_duplicates() \
               .sort_values(by = ['Country', 'State', 'City'])

# Region input options: Africa, Asia, Australia/South Pacific, Europe, Middle East, North America, South/Central America & Carribean
unique_values(Region = 'South/Central America & Carribean')

,Region,Country,State,City
1231460,South/Central America & Carribean,Argentina,NaN,Buenos Aires
1240726,South/Central America & Carribean,Bahamas,NaN,Nassau
1274233,South/Central America & Carribean,Barbados,NaN,Bridgetown
1255704,South/Central America & Carribean,Belize,NaN,Belize City
1249991,South/Central America & Carribean,Bermuda,NaN,Hamilton
1264967,South/Central America & Carribean,Bolivia,NaN,La Paz
1282775,South/Central America & Carribean,Brazil,NaN,Brasilia
1292041,South/Central America & Carribean,Brazil,NaN,Rio de Janeiro
1301306,South/Central America & Carribean,Brazil,NaN,Sao Paulo
1310572,South/Central America & Carribean,Colombia,NaN,Bogota


In [13]:
# Check if columns contain null values. Only state has nulls, which is to be expected.
temp_df.loc[pd.isnull(temp_df[['Region', 'Country', 'City', 'Date', 'AvgTemperature']]).any(1),:]

,Region,Country,State,City,Month,Day,Year,AvgTemperature,Date


#### Population Counts by Country and Year

In [14]:
# Rename columns to be more descriptive
country_pop_df.columns = ['Country', 'Year', 'Country_Population']

# Check how consistent information is populated by country over time
country_pop_df.groupby(['Year'])[['Country']].nunique()
# Very consistent going back to 1960

,Country
Year,
1960,216
1961,216
1962,216
1963,216
1964,216
1965,216
1966,216
1967,216
1968,216


In [16]:
# Summary stats by country. 
country_pop_df.where(country_pop_df.Country > 'C') \
    .groupby(['Country']) \
    .agg(['min', 'max', 'nunique'])
# country_pop_df.agg(['min', 'max', 'nunique'])
# country_pop_df.where(country_pop_df.Country == 'China')

Year                 Country_Population  \
                             min     max nunique                min   
Country                                                               
Cabo Verde                1960.0  2017.0      58           201765.0   
Cambodia                  1960.0  2017.0      58          5722370.0   
Cameroon                  1960.0  2017.0      58          5176918.0   
Canada                    1960.0  2017.0      58         17909009.0   
Caribbean small states    1960.0  2017.0      58          4194710.0   
Cayman Islands            1960.0  2017.0      58             7865.0   
Central African Republic  1960.0  2017.0      58          1501668.0   
Chad                      1960.0  2017.0      58          3001609.0   
Channel Islands           1960.0  2017.0      58           109420.0   
Chile                     1960.0  2017.0      58          8132990.0   
China                     1960.0  2017.0      58        660330000.0   
Colombia                  1960.0  2017.0      58         16057724.0   
Comoros                   1960.0  2017.0      58           191121.0   
Congo, Dem. Rep.          1960.0  2017.0      58         15248251.0   
Congo, Rep.               1960.0  2017.0      58          1018253.0   
Costa Rica                1960.0  2017.0      58          1330782.0   
Cote d'Ivoire             1960.0  2017.0      58          3503553.0   
Croatia                   1960.0  2017.0      58          4124531.0   
Cuba                      1960.0  2017.0      58          7141250.0   
Curacao                   1960.0  2017.0      58           124826.0   
Cyprus                    1960.0  2017.0      58           572930.0   
Czech Republic            1960.0  2017.0      58          9586651.0   
Denmark                   1960.0  2017.0      58          4579603.0   
Djibouti                  1960.0  2017.0      58            83636.0   
Dominica                  1960.0  2017.0      58            60011.0   
Dominican Republic        1960.0  2017.0      58          3294224.0   
Ecuador                   1960.0  2017.0      58          4543666.0   
Egypt, Arab Rep.          1960.0  2017.0      58         26632894.0   
El Salvador               1960.0  2017.0      58          2766324.0   
Equatorial Guinea         1960.0  2017.0      58           239678.0   
...                          ...     ...     ...                ...   
Sweden                    1960.0  2017.0      58          7484656.0   
Switzerland               1960.0  2017.0      58          5327827.0   
Syrian Arab Republic      1960.0  2017.0      58          4573512.0   
Tajikistan                1960.0  2017.0      58          2087049.0   
Tanzania                  1960.0  2017.0      58         10052152.0   
Thailand                  1960.0  2017.0      58         27397207.0   
Timor-Leste               1960.0  2017.0      58           474532.0   
Togo                      1960.0  2017.0      58          1580513.0   
Tonga                     1960.0  2017.0      58            61577.0   
Trinidad and Tobago       1960.0  2017.0      58           847979.0   
Tunisia                   1960.0  2017.0      58          4178236.0   
Turkey                    1960.0  2017.0      58         27472345.0   
Turkmenistan              1960.0  2017.0      58          1603258.0   
Turks and Caicos Islands  1960.0  2017.0      58             5744.0   
Tuvalu                    1960.0  2017.0      58             5323.0   
Uganda                    1960.0  2017.0      58          6767095.0   
Ukraine                   1960.0  2017.0      58         42664652.0   
United Arab Emirates      1960.0  2017.0      58            92418.0   
United Kingdom            1960.0  2017.0      58         52400000.0   
United States             1960.0  2017.0      58        180671000.0   
Uruguay                   1960.0  2017.0      58          2538643.0   
Uzbekistan                1960.0  2017.0      58          8526300.0   
Vanuatu                   1960.0  2017.0      58    

In [17]:
# Verify there are no duplicate records at the most granular level, which is country and year
# No duplicates.
duplicate = country_pop_df[country_pop_df.duplicated(['Country', 'Year'], keep=False)] \
                .sort_values(by = ['Country', 'Year'])
duplicate.head()

,Country,Year,Country_Population


In [18]:
# Review country names for inconsistencies. May have adjust names to align with other sources.
country_pop_df[['Country']].drop_duplicates() \
    .sort_values(by = ['Country'])

,Country
1,Afghanistan
3,Albania
56,Algeria
8,American Samoa
4,Andorra
2,Angola
9,Antigua and Barbuda
6,Argentina
7,Armenia
0,Aruba


In [19]:
# Check if columns contain null values. No null values detected.
country_pop_df.loc[pd.isnull(country_pop_df[['Country', 'Year', 'Country_Population']]).any(1),:]

,Country,Year,Country_Population


#### Population Counts by City and Year

In [20]:
# Filter on sex to only include both sexes since other sources don't include this breakdown.
# Note I verfied there are 4,751 distinct cities and all cities have a both sexes row
city_pop_df[['City']].nunique()
city_pop_df[['Sex','City']].groupby(['Sex']).nunique()
city_pop_df = city_pop_df[city_pop_df.Sex == 'Both Sexes']

# By removing the sex breakdown, the field can be dropped
city_pop_df = city_pop_df.drop(columns=['Sex'])
# city_pop_df.head()

# Check how many inputs are in the Area column
city_pop_df[['Area']].drop_duplicates()

# Remove Area coulmn with there being only one input
city_pop_df = city_pop_df.drop(columns=['Area'])

# Rename columns for naming consistencies
city_pop_df.columns = ['Country_or_Area',
                       'Year',
                       'City',
                       'City_Type',
                       'Record_Type',
                       'Reliability',
                       'Source_Year',
                       'City_Population',
                       'Population_Notes']

# Summary stats by country/area and sex. There are gaps in time.
city_pop_df[['Country_or_Area', 'Year', 'City_Population']] \
    .groupby(['Country_or_Area']) \
    .agg(['min', 'max', 'nunique'])

Year                \
                                                     min   max nunique   
Country_or_Area                                                          
Albania                                             2003  2011       2   
Algeria                                             1998  2008       2   
American Samoa                                      2000  2010       2   
Andorra                                             2003  2011       8   
Anguilla                                            2001  2011       2   
Antigua and Barbuda                                 1991  1991       1   
Argentina                                           1991  2020       8   
Armenia                                             2001  2018      11   
Aruba                                               1991  2010       2   
Australia                                           2001  2018      17   
Austria                                             2001  2019      15   
Azerbaijan                                          2003  2019      12   
Bahamas                                             2000  2016       3   
Bahrain                                             2001  2001       1   
Bangladesh                                          1991  2011       3   
Barbados                                            1980  1980       1   
Belarus                                             1999  2018      13   
Belgium                                             2000  2011       6   
Belize                                              2010  2010       1   
Benin                                               2002  2013       2   
Bermuda                                             2000  2016       3   
Bhutan                                              2005  2017       2   
Bolivia (Plurinational State of)                    2001  2001       1   
Bosnia and Herzegovina                              1991  1991       1   
Botswana                                            2001  2011       2   
Brazil                                              2010  2010       1   
British Virgin Islands                              1992  1992       1   
Brunei Darussalam                                   2001  2001       1   
Bulgaria                                            2003  2019      14   
Burkina Faso                                        1996  2006       2   
...                                                  ...   ...     ...   
Sri Lanka                                           2001  2001       1   
State of Palestine                                  2007  2017       2   
Suriname                                            2003  2012       3   
Sweden                                              1999  2007       3   
Switzerland                                         2000  2018      12   
Tajikistan                                          1993  2013       6   
Thailand                                            2000  2010       2   
Timor-Leste                                         2010  2015       2   
Togo                                                2010  2010       1   
Tonga                                               2011  2016       2   
Trinidad and Tobago                                 1996  2011       2   
Tunisia                                             2014  2014       1   
Turkmenistan                                        1990  1990       1   
Turks and Caicos Islands                            1990  2012       2   
Tuvalu                                              1991  2012       3   
Uganda                                              2002  2014       2   
Ukraine                                             2001  2018      10   
United Kingdom of Great Britain and Northern Ir...  2001  2011       2   
United Republic of Tanzania                         1988  2012       2   
United States Virgin Islands                        2000  2010       2   
United States of America                            2000

In [21]:
# Drop duplicates
city_pop_df = city_pop_df.drop_duplicates()

# Verify there are no duplicate records at the most granular level
# There are duplicates. Need to prioritize most recent source year, preference on record type (census over estimate), and maybe reliability.
duplicate = city_pop_df[city_pop_df.duplicated(['City', 'Year', 'City_Type'], keep=False)] \
                .sort_values(by = ['City', 'Year'])
duplicate

,Country_or_Area,Year,City,City_Type,Record_Type,Reliability,Source_Year,City_Population,Population_Notes
53989,Spain,2001,A Coruña,City proper,Census - de facto - complete tabulation,"Final figure, complete",2009,236379.0,NaN
53990,Spain,2001,A Coruña,City proper,Estimate - de jure,"Final figure, complete",2002,235847.0,170
52883,Spain,2011,A Coruña,City proper,Census - de jure - complete tabulation,"Final figure, complete",2014,245055.0,74
52884,Spain,2011,A Coruña,City proper,Estimate - de jure,"Final figure, complete",2013,246087.0,74
32822,Kazakhstan,2012,ASTANA,City proper,Estimate - de facto,"Final figure, complete",2018,760506.0,"18,118"
32823,Kazakhstan,2012,ASTANA,City proper,Estimate - de facto,"Final figure, complete",2015,778198.0,NaN
32630,Kazakhstan,2013,ASTANA,City proper,Estimate - de facto,"Final figure, complete",2018,796282.0,"18,118"
32631,Kazakhstan,2013,ASTANA,City proper,Estimate - de facto,"Final figure, complete",2015,814435.0,NaN
18183,Germany,2011,Aachen,City proper,Census - de jure - complete tabulation,"Final figure, complete",2014,236420.0,NaN
18184,Germany,2011,Aachen,City proper,Estimate - de jure,"Final figure, complete",2011,258664.0,NaN


In [22]:
# Check how consistent information is populated by city over time
city_pop_df.groupby(['Year'])[['City']].nunique()
# By year 2000, data collection is much more complete accross cities

,City
Year,
1970,2
1976,9
1980,3
1981,1
1983,9
1984,4
1985,2
1986,2
1987,28


In [23]:
# Review country and city names for inconsistencies. 
city_pop_df[['Country_or_Area', 'City']] \
    .drop_duplicates() \
    .sort_values(by = ['Country_or_Area'])
# Will have adjust names to align with other sources.

,Country_or_Area,City
42,Albania,Durrës
43,Albania,TIRANA
86,Algeria,Souq Ahras
78,Algeria,Mouaskar (Mascara)
79,Algeria,M'Sila
81,Algeria,Oum El Bouaghi
82,Algeria,Qacentina (Constantine)
83,Algeria,Saïda
84,Algeria,Sidi-bel-Abbès
85,Algeria,Skikda


In [24]:
# Review record types and reliability types to familiarize with. 
city_pop_df[['Record_Type', 'Reliability',]] \
    .drop_duplicates() \
    .sort_values(by = ['Record_Type'])
# May have prioirtize when there are multiple sources for the same year and city

,Record_Type,Reliability
157,Census - de facto - complete tabulation,"Final figure, complete"
2678,Census - de facto - complete tabulation,Provisional figure
39,Census - de jure - complete tabulation,"Final figure, complete"
34434,Census - de jure - complete tabulation,Provisional figure
15932,Census - de jure - sample tabulation,"Final figure, complete"
48,Estimate - de facto,"Final figure, complete"
2567,Estimate - de facto,Provisional figure
0,Estimate - de jure,"Final figure, complete"
764,Estimate - de jure,Provisional figure
34484,Estimate - de jure,Other estimate


In [25]:
# Check if columns contain null values. 
city_pop_df.loc[pd.isnull(city_pop_df[['Country_or_Area',
                                       'Year',
                                       'City',
                                       'City_Type',
                                       'Record_Type', 
                                       'Reliability',
                                       'Source_Year', 
                                       'City_Population']]).any(1),:]
# No unexpected nulls.

,Country_or_Area,Year,City,City_Type,Record_Type,Reliability,Source_Year,City_Population,Population_Notes


#### Refugee Counts by Year, Country of Residence, and Country of Origin

In [26]:
# Rename columns for naming consistencies
refugee_df.columns = ['Asylum_Country_or_Territory',
                      'Origin_Country_or_Territory',
                      'Year',
                      'Refugees',
                      'Refugees_Assisted_by_UNHCR',
                      'Refugee-like_Population',
                      'Refugee-like_Population_Assisted_by_UNHCR']

# Drop duplicates
refugee_df = refugee_df.drop_duplicates()

# Get an understanding of the differences between population counts based on summary stats
def pop_stat_comparison(pop1, pop2):
    return refugee_df[['Asylum_Country_or_Territory', pop1, pop2]] \
               .groupby(['Asylum_Country_or_Territory']) \
               .agg(['min', 'max', 'nunique'])

pop_stat_comparison('Refugees', 'Refugees_Assisted_by_UNHCR')
# pop_stat_comparison('Refugee-like_Population', 'Refugee-like_Population_Assisted_by_UNHCR')
# pop_stat_comparison('Refugee-like_Population', 'Refugees')

# Main takeaways:
# UNHCR related counts are populated substantially less
# Refugees vs. refugee-like: they differ slightly

Refugees                     \
                                           min        max nunique   
Asylum_Country_or_Territory                                         
Afghanistan                                1.0   280229.0      29   
Albania                                    1.0    22324.0      37   
Algeria                                    1.0   165000.0      82   
Angola                                     1.0   225000.0     156   
Anguilla                                   1.0        1.0       1   
Antigua and Barbuda                        4.0       15.0       2   
Argentina                                  1.0    38320.0     170   
Armenia                                    1.0   328000.0      69   
Aruba                                      1.0        1.0       1   
Australia                                  1.0   317000.0     588   
Austria                                    1.0    74131.0     448   
Azerbaijan                                 1.0   201440.0      85   
Bahamas                                    1.0      374.0      14   
Bahrain                                    1.0     2000.0      21   
Bangladesh                                 1.0   244994.0      62   
Barbados                                   1.0        1.0       1   
Belarus                                    1.0    30525.0      74   
Belgium                                    1.0    36600.0     463   
Belize                                     1.0    25300.0      84   
Benin                                      1.0   155908.0     171   
Bolivia (Plurinational State of)           1.0      632.0      60   
Bosnia and Herzegovina                     1.0    40000.0      51   
Botswana                                   1.0    22530.0     174   
Brazil                                     1.0    36000.0     224   
British Virgin Islands                     2.0        2.0       1   
Bulgaria                                   1.0    15027.0     166   
Burkina Faso                               1.0    49500.0     111   
Burundi                                    1.0   278092.0     109   
Cabo Verde                                 2.0        3.0       2   
Cambodia                                   1.0      357.0      29   
...                                        ...        ...     ...   
Swaziland                                  1.0    48054.0     130   
Sweden                                     1.0   139783.0     856   
Switzerland                                1.0    67061.0     644   
Syrian Arab Rep.                           1.0  1500000.0     205   
Tajikistan                                 1.0    15354.0      37   
Thailand                                   1.0   147400.0     222   
The former Yugoslav Rep. of Macedonia      1.0    32000.0      40   
Timor-Leste                                1.0        2.0       2   
Togo                                       1.0    18476.0     132   
Tonga                                      1.0        2.0       2   
Trinidad and Tobago                        1.0       40.0      14   
Tunisia                                    1.0      953.0      70   
Turkey                                     1.0  2823987.0     227   
Turkmenistan                               1.0    20000.0      58   
Turks and Caicos Islands                   1.0        3.0       2   
Uganda                                     1.0   639007.0     249   
Ukraine                                    1.0     5193.0     174   
United Arab Emirates                       1.0    20000.0      67   
United Kingdom                             1.0   153000.0     965   
United Rep. of Tanzania                    1.0   626196.0     205   
United States                              1.0   734000.0    1627   
Uruguay                                    1.0     2000.0      55   
Uzbekistan                                 1.0    39202.0      39   
Vanuatu                                    1.0        3.0       2   
Various                             

In [28]:
# Summary stats by country of asylum.
refugee_df[['Asylum_Country_or_Territory', 'Year']] \
    .groupby(['Asylum_Country_or_Territory']) \
    .agg(['min', 'max', 'nunique'])
# There are gaps in time.

Year              
                                        min   max nunique
Asylum_Country_or_Territory                              
Afghanistan                            1990  2016      24
Albania                                1992  2016      25
Algeria                                1975  2016      42
Angola                                 1976  2016      41
Anguilla                               2015  2016       2
Antigua and Barbuda                    2015  2016       2
Argentina                              1975  2016      42
Armenia                                1992  2016      25
Aruba                                  2013  2016       3
Australia                              1978  2016      39
Austria                                1975  2016      42
Azerbaijan                             1992  2016      25
Bahamas                                1992  2016      15
Bahrain                                1985  2016      18
Bangladesh                             1978  2016      28
Barbados                               2013  2014       2
Belarus                                1994  2016      23
Belgium                                1975  2016      42
Belize                                 1980  2015      36
Benin                                  1983  2016      34
Bolivia (Plurinational State of)       1978  2016      36
Bosnia and Herzegovina                 1997  2016      20
Botswana                               1975  2016      42
Brazil                                 1975  2016      42
British Virgin Islands                 2010  2012       3
Bulgaria                               1992  2016      25
Burkina Faso                           1983  2016      34
Burundi                                1975  2016      42
Cabo Verde                             1991  1992       2
Cambodia                               1993  2016      24
...                                     ...   ...     ...
Swaziland                              1975  2016      42
Sweden                                 1975  2016      42
Switzerland                            1975  2016      42
Syrian Arab Rep.                       1978  2016      37
Tajikistan                             1992  2016      25
Thailand                               1975  2016      42
The former Yugoslav Rep. of Macedonia  1992  2016      25
Timor-Leste                            2002  2010       9
Togo                                   1976  2016      36
Tonga                                  2011  2013       3
Trinidad and Tobago                    2007  2016      10
Tunisia                                1990  2016      27
Turkey                                 1975  2016      42
Turkmenistan                           1993  2016      24
Turks and Caicos Islands               2013  2016       4
Uganda                                 1975  2016      42
Ukraine                                1994  2016      23
United Arab Emirates                   1976  2016      35
United Kingdom                         1975  2016      42
United Rep. of Tanzania                1975  2016      42
United States                          1975  2016      42
Uruguay                                1977  2016      39
Uzbekistan                             1993  2016      24
Vanuatu                                2007  2013       6
Various                                1975  1992      18
Venezuela (Bolivarian Republic of)     1977  2016      40
Viet Nam                               1977  2011      35
Yemen                                  1977  2016      40
Zambia                                 1975  2016      42
Zimbabwe                               1980  2016      37

[190 rows x 3 columns]

In [29]:
# Verify there are no duplicate records at the most granular level
duplicate = refugee_df[refugee_df.duplicated(['Asylum_Country_or_Territory',
                                              'Origin_Country_or_Territory',
                                              'Year'], keep=False)] \
                           .sort_values(by = ['Asylum_Country_or_Territory',
                                              'Origin_Country_or_Territory',
                                              'Year'])
duplicate
# No duplicates

,Asylum_Country_or_Territory,Origin_Country_or_Territory,Year,Refugees,Refugees_Assisted_by_UNHCR,Refugee-like_Population,Refugee-like_Population_Assisted_by_UNHCR


In [30]:
# Check how consistent information is populated by city over time
refugee_df.groupby(['Year'])[['Asylum_Country_or_Territory']].nunique()
# Seems consistent by year, especially considering new countries forming over time

,Asylum_Country_or_Territory
Year,
1975,50
1976,53
1977,72
1978,82
1979,88
1980,90
1981,92
1982,94
1983,97


In [31]:
# Review country and city names for inconsistencies. 
refugee_df[['Asylum_Country_or_Territory', 'Origin_Country_or_Territory']] \
    .drop_duplicates() \
    .sort_values(by = ['Asylum_Country_or_Territory'])
# May have adjust names to align with other sources.

,Asylum_Country_or_Territory,Origin_Country_or_Territory
0,Afghanistan,Iraq
35389,Afghanistan,Eritrea
10777,Afghanistan,State of Palestine
83439,Afghanistan,Tajikistan
21052,Afghanistan,Syrian Arab Rep.
90464,Afghanistan,Various
2,Afghanistan,Pakistan
1,Afghanistan,Islamic Rep. of Iran
83444,Albania,Sri Lanka
83440,Albania,Bosnia and Herzegovina


In [33]:
# Check if columns contain null values. 
refugee_df.loc[pd.isnull(refugee_df[['Asylum_Country_or_Territory',
                                     'Origin_Country_or_Territory',
                                     'Year',
                                     'Refugees',
                                     'Refugee-like_Population']]).any(1),:]
# No unexpected nulls. Not all refugee counts populate, which is okay.

,Asylum_Country_or_Territory,Origin_Country_or_Territory,Year,Refugees,Refugees_Assisted_by_UNHCR,Refugee-like_Population,Refugee-like_Population_Assisted_by_UNHCR
2453,Israel,Dem. Rep. of the Congo,2016,NaN,NaN,208.0,50.0
2941,Malaysia,Kenya,2016,NaN,NaN,1.0,1.0
2953,Malaysia,Rep. of Moldova,2016,NaN,NaN,1.0,1.0
2966,Malaysia,United States,2016,NaN,NaN,1.0,1.0
2967,Malaysia,Viet Nam,2016,NaN,NaN,1.0,1.0
4043,Saudi Arabia,Liberia,2016,NaN,NaN,7.0,7.0
4091,Serbia (and Kosovo: S/RES/1244 (1999)),Various,2016,NaN,NaN,1150.0,1150.0
7889,Israel,Dem. Rep. of the Congo,2015,NaN,NaN,208.0,50.0
8361,Malaysia,Kenya,2015,NaN,NaN,1.0,1.0
9441,Saudi Arabia,Liberia,2015,NaN,NaN,7.0,7.0


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.